# Injecting Secrets into Kubernetes Pods via Vault Agent Containers
- reference link: https://developer.hashicorp.com/vault/tutorials/kubernetes/kubernetes-sidecar

### Set a secret in Vault
- we using key value engine
- execute command inside container with -it flag
    - kubectl exec -it vault-0 -- /bin/sh
- enable key value secret engine with specific path
    - vault secrets enable -path=internal kv-v2
- using vault kv put command to create secret
    - vault kv put internal/database/config username="db-readonly-username" password="db-secret-password"
- check secret have been created with command vault kv get
    - vault kv get internal/database/config
- exit interactive pod with command exit

### Configure Kubernetes authentication
- config k8s authentication so that k8s pods can talk to vault pod
- execute vault command with interactive mode with -it flag
    - kubectl exec -it $pod_name -- /bin/sh
- enalbe k8s authenticate in vault
    - vault auth enable kubernetes
- config k8s api service address
    - vault write auth/kubernetes/config \
      kubernetes_host="https://$KUBERNETES_PORT_443_TCP_ADDR:443"
- config policy
    - policy is object which define permission to secret
    - vault policy write internal-app - <<EOF
        path "internal/data/database/config" {
        capabilities = ["read"]
        }
        EOF
- config role
    - role is an object which binding k8s service account with role
        - a biding between who (service account) with role (permission on resource or secret)
    - vault write auth/kubernetes/role/internal-app \
        bound_service_account_names=internal-app \
        bound_service_account_namespaces=default \
        policies=internal-app \
        ttl=24h
- exit current interactive mode with command: exit

### Define a Kubernetes service account
- list service account
    - kubectl get sa
- create service account
    - kubectl create sa $sa_name
    - example:
        - kubectl create sa internal-app
- check service account have been create with get
    - kubectl get sa


### Launch an application
- config deployment at ./config/deployment-orgchart.yaml
- apply deployment with kubectl apply command:
    - kubectl apply -f ./config/deployment-orgchart.yaml
- check that secret have been inject to pod or not
    - kubectl exec \
      $(kubectl get pod -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
      --container orgchart -- ls /vault/secrets
- execute command inside container
    - kubectl exec $pod_name --contaiern $container_name -- $command_line
- note for checking injected secret
    - we have define deployment and would like vault agent inject to inject secret into that deployment pods
    - we check by using kubectl exec command to go inside that pod's container and run command ls


### Inject secrets into the pod
- define a patch (change) for orchart deployent
    - ./config/patch-inject-secrets.yaml
- apply patch
    - kubectl patch deployment orgchart --patch "$(cat ./config/patch-inject-secrets.yaml)"
- note after apply patch to deployment
    - after apply patch to deployment, k8s will put additional vault-agent container into the pod
    - so that vault-agent can write secrets into shared volume (/vault_secrets)
    - now orgchart can access shared volume which stored secrets
- check logs at vault-agent container inside orchart pods with kubectl logs command
    - kubectl logs \
        $(kubectl get pod -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
        --container vault-agent
- check secret which orgchart container on orchart pod can have access
    - kubectl exec \
        $(kubectl get pod -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
        --container orgchart -- cat /vault/secrets/database-config.txt



### Apply a template to the injected secrets
- config patch so that secrets will write with template
- the path config is define at: ./config/patch-inject-secrets-as-template.yaml
- apply template path
    - kubectl patch deployment orgchart --patch "$(cat ./config/patch-inject-secrets-as-template.yaml)"
- check pod with kubectl get pods
    - kubectl get pods
- check the secret have been inject with template or not
    - kubectl exec \
      $(kubectl get pod -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
      -c orgchart -- cat /vault/secrets/database-config.txt
    - flag -c is for container


### Pod with annotations
- we can inject secret to pod with Pod object just like development object through annotation
- reference link about vault injection annotations: https://developer.hashicorp.com/vault/docs/platform/k8s/injector/annotations


### Secrets are bound to the service account and namespace
- the service account which use by Deployment or Pod have to be the same with service account config in vault role
- the namespace which specify in Deployment or Pod have to be the same with name space config in vault role
- Service account in k8s
    - service account is non-human account to indentify pods or group of pods
    - serice account usually assign to deployment or statefulset to idenity pods belong to that Deployment or StatefulSet
- namespace in k8s
    - namespace is isolate other object into a workspace for easy management and isolation of resource
